# 📊 Teoría del RSI (Relative Strength Index)

El **RSI (Relative Strength Index)** es un oscilador de momentum desarrollado por J. Welles Wilder (1978).  
Su objetivo es medir la **fuerza relativa** de los movimientos de precios, identificando condiciones de **sobrecompra** y **sobreventa**.

---

## 🔹 Fórmula del RSI

El RSI se basa en la relación entre las ganancias y pérdidas promedio durante un número de períodos ($n$):

1. **Cambio de precios (Δ):**

$$
\Delta P_t = P_t - P_{t-1}
$$

2. **Ganancias y pérdidas promedio (método de Wilder):**

$$
\text{Ganancia Promedio} = \frac{\sum_{i=1}^n \max(\Delta P_i, 0)}{n}
$$

$$
\text{Pérdida Promedio} = \frac{\sum_{i=1}^n \max(-\Delta P_i, 0)}{n}
$$

3. **Índice de Fuerza Relativa (RS):**

$$
RS_t = \frac{\text{Ganancia Promedio}}{\text{Pérdida Promedio}}
$$

4. **RSI:**

$$
RSI_t = 100 - \frac{100}{1 + RS_t}
$$

---

## 🔹 Interpretación del RSI

- Valores cercanos a **70** → condición de **sobrecompra** (potencial corrección o baja).  
- Valores cercanos a **30** → condición de **sobreventa** (potencial rebote o alza).  
- Valores alrededor de **50** → mercado equilibrado, sin tendencia clara.

---

## 🔹 Señales de Trading

1. **Sobrecompra/Sobreventa:**
   - RSI > 70 → posible presión vendedora (señal de venta).  
   - RSI < 30 → posible presión compradora (señal de compra).  

2. **Cruces con el nivel 50:**
   - RSI > 50 → tendencia alcista.  
   - RSI < 50 → tendencia bajista.  

3. **Divergencias:**
   - Si el precio hace nuevos máximos, pero el RSI no lo confirma → divergencia bajista.  
   - Si el precio hace nuevos mínimos, pero el RSI no lo confirma → divergencia alcista.  

---

## 🔹 Parámetros típicos

- El período más común es **14** (RSI-14).  
- Puede ajustarse:
  - Corto plazo → 7 períodos (más sensible).  
  - Largo plazo → 21 períodos (más estable).  

---

## 📊 Usos comunes

- Confirmación de tendencias.  
- Detección de niveles extremos de mercado.  
- Identificación de divergencias precio–momentum.  

---

In [1]:
# !pip install yfinance plotly

import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ------------------ RSI (Wilder) ------------------
def compute_rsi(close: pd.Series, period: int = 14) -> pd.Series:
    delta = close.diff()
    gain = np.where(delta > 0, delta, 0.0)
    loss = np.where(delta < 0, -delta, 0.0)

    # Suavizado tipo Wilder (EMA con alpha = 1/period)
    avg_gain = pd.Series(gain, index=close.index).ewm(alpha=1/period, adjust=False).mean()
    avg_loss = pd.Series(loss, index=close.index).ewm(alpha=1/period, adjust=False).mean()

    rs = avg_gain / (avg_loss.replace(0, np.nan))
    rsi = 100 - (100 / (1 + rs))
    rsi = rsi.fillna(50)  # neutral para los primeros valores
    return rsi

# ------------------ Gráfico velas + RSI ------------------
def plot_candles_rsi(
    ticker: str,
    start_date: str,
    end_date: str,
    rsi_period: int = 14
):
    df = yf.download(ticker, start=start_date, end=end_date, auto_adjust=False, progress=False)
    if df.empty:
        raise ValueError(f"Sin datos para {ticker} en {start_date} → {end_date}")

    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    df = df[['Open','High','Low','Close']].dropna().copy()

    rsi = compute_rsi(df['Close'], period=rsi_period)

    # Subplots 50% / 50%
    fig = make_subplots(
        rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05,
        row_heights=[0.5, 0.5],
        subplot_titles=(f"{ticker} | Velas", f"RSI ({rsi_period})")
    )

    # --- Velas (verde/rojo) ---
    fig.add_trace(go.Candlestick(
        x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'],
        increasing_line_color='green', increasing_fillcolor='green',
        decreasing_line_color='red', decreasing_fillcolor='red',
        name=ticker
    ), row=1, col=1)

    # --- RSI (línea morada) ---
    fig.add_trace(go.Scatter(
        x=df.index, y=rsi, mode="lines", name=f"RSI {rsi_period}",
        line=dict(color="purple", width=2)
    ), row=2, col=1)

    # Zonas y líneas: Overbought (70, rojo) y Oversold (30, verde)
    fig.add_hrect(y0=70, y1=100, line_width=0, fillcolor="rgba(255,0,0,0.10)", row=2, col=1)
    fig.add_hrect(y0=0,  y1=30,  line_width=0, fillcolor="rgba(0,128,0,0.10)", row=2, col=1)

    fig.add_hline(y=70, line_color="red",  line_dash="dash", row=2, col=1)
    fig.add_hline(y=30, line_color="green", line_dash="dash", row=2, col=1)

    # Ejes y layout
    fig.update_yaxes(title_text="Precio", row=1, col=1)
    fig.update_yaxes(title_text="RSI",   row=2, col=1, range=[0, 100])

    fig.update_layout(
        title=f"{ticker} | Velas + RSI ({start_date} → {end_date})",
        xaxis_rangeslider_visible=False,
        hovermode="x unified",
        template="plotly_white",
        margin=dict(l=90, r=90, t=60, b=40),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )

    fig.show()


In [2]:
#@title Seleccione Acción y Fecha {run:'auto'}
ticker = "NVDA" #@param ["AAPL", "MSFT", "NVDA", "GOOGL", "AMZN", "META", "TSM","BRK.B", "V", "JPM", "XOM", "LLY", "MRK", "UNH", "PG", "MA","CVX", "KO", "PEP", "COST", "TMO", "ORCL", "CSCO", "NKE", "VZ", "ASML", "TXN", "ABT", "TM", "SAP", "AMD", "NFLX", "NOW", "ADBE", "LVMUY", "BABA", "SHEL", "TMUS", "QCOM", "PFE", "SNY", "AZN", "TOT", "GSK", "RIO", "BHP", "MCD​"]
#ticker = "BTC-USD" #@param ["BTC-USD", "ETH-USD", "USDT-USD", "XRP-USD", "LTC-USD", "ADA-USD", "DOT-USD", "BCH-USD", "XLM-USD", "LINK-USD"]

start_date = '2024-09-09' #@param {type:'date'}
end_date = '2025-09-09' #@param {type:'date'}




In [3]:
plot_candles_rsi(
    ticker, start_date, end_date,
    rsi_period=14  #Intenta a 21 suavizante
)

In [4]:
3# Cómo calcular las ganancias con esta estrategia
inversion_inicial = 10_000.00 # usd
# Compras
# 1
oversold1 = 94.31 #abril 4 - aqui compre
acciones1 = inversion_inicial / oversold1
overbought1 = 135.34 #Mayo 14 - aqui vendi
ganancia1 = (acciones1 * overbought1) - inversion_inicial
valorFinal1 = inversion_inicial + ganancia1
#
print(f"Inversion Inicial ${inversion_inicial:,.2f}")
print(f"Acciones Compradas {acciones1:,.2f}")
print(f"Sobre Venta ${oversold1:,.2f}")
print(f"Sobre Compra ${overbought1:,.2f}")
print(f"Valor Final ${valorFinal1:,.2f}")
print(f"Ganancia primera compra-venta ${ganancia1:,.2f} que es {ganancia1/inversion_inicial * 100:,.2f}%")

Inversion Inicial $10,000.00
Acciones Compradas 106.03
Sobre Venta $94.31
Sobre Compra $135.34
Valor Final $14,350.55
Ganancia primera compra-venta $4,350.55 que es 43.51%


In [5]:
# 2
inversion_inicial_2 = valorFinal1
oversold2 = 131.29  # mayo 23 - aqui compramos nuevamente
acciones2 = inversion_inicial_2 / oversold2 # aqui reinvertimos las ganancias y el capital inicial
overbought2 = 157.99 # Junio 30 - aqui vendemos
ganancia2 = (acciones2 * overbought2) - inversion_inicial_2
valorFinal2 = inversion_inicial_2 + ganancia2
#
print(f"Inversion Inicial ${inversion_inicial_2:,.2f}")
print(f"Acciones Compradas {acciones2:,.2f}")
print(f"Sobre Venta ${oversold2:,.2f}")
print(f"Sobre Compra ${overbought2:,.2f}")
print(f"Valor Final ${valorFinal2:,.2f}")
print(f"Ganancia segunda compra-venta: ${ganancia2:,.2f} que es {ganancia2/inversion_inicial_2 * 100:,.2f}%")

Inversion Inicial $14,350.55
Acciones Compradas 109.30
Sobre Venta $131.29
Sobre Compra $157.99
Valor Final $17,268.97
Ganancia segunda compra-venta: $2,918.42 que es 20.34%


In [6]:
# Reporte Final
rendimiento = (valorFinal2 - inversion_inicial) / inversion_inicial
print(f"Valor Final despues de dos operaciones: ${valorFinal2:,.2f}")
print(f"Rendimiento {rendimiento:,.2%}")

Valor Final despues de dos operaciones: $17,268.97
Rendimiento 72.69%
